In [2]:
import numpy as np
import time

import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torch.autograd as autograd
import numpy as np

syft = sy 

hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
james = sy.VirtualWorker(hook, id="james")
crypto_provider = james
torch = th

from syft.serde.msgpack import serde

In [3]:
λ = 6 #6 #63 # security paramater
n = 8 #8 #32 # nb of bits of the algebric field

# DPF

In [4]:
# PRG
def G(seed):
    #OK
    assert len(seed) == λ
    th.manual_seed(Convert(seed))
    return th.randint(2, size=(2*(λ + 1),), dtype=th.uint8)

#bit_pow_lambda = th.flip(2 ** th.arange(λ), (0,)).to(th.uint8)
def Convert(bits):
    bit_pow_lambda = th.flip(2 ** th.arange(λ), (0,)).to(th.uint8)
    return bits.dot(bit_pow_lambda)

def Array(*shape):
    #FIXED SHAPE
    return th.empty(shape, dtype=th.uint8)

#bit_pow_n = th.flip(2 ** th.arange(n), (0,))
def bit_decomposition(x):
    bit_pow_n = th.flip(2 ** th.arange(n), (0,))
    return ((x & bit_pow_n) > 0).to(th.int8)

def randbit(size):
    #FIXED SHAPE
    return th.randint(2, size=size)
    
def concat(*args, **kwargs):
    return th.cat(args, **kwargs)

def split(x, idx):
    return th.split(x, idx)
    
#one = th.tensor([1], dtype=th.uint8)
def TruthTableDPF(s, α_i):
    one = th.tensor([1], dtype=th.uint8)
    Table = th.zeros((2, λ+1), dtype=th.uint8)
    Table[α_i] = concat(s, one)
    return Table.flatten()

In [5]:
@sy.func2plan()
def GenDPF(alpha, beta):
    α = bit_decomposition(alpha)
    s, t, CW = Array(n+1, 2, λ), Array(n+1, 2), Array(n, 2*(λ+1))
    s[0] = randbit(size=(2, λ))
    t[0] = th.tensor([0, 1], dtype=torch.uint8)
    for i in range(0, n):
        # Re-use useless randomness
        sL_0, _, sR_0, _ = split(G(s[i, 0]), [λ, 1, λ, 1])
        sL_1, _, sR_1, _ = split(G(s[i, 1]), [λ, 1, λ, 1])
        s_rand = (sL_0 ^ sL_1) * α[i] + (sR_0 ^ sR_1) * (1 - α[i])
        
        cw_i = TruthTableDPF(s_rand, α[i])
        CW[i] = cw_i ^ G(s[i, 0]) ^ G(s[i, 1])
        
        for b in (0, 1):
            τ = G(s[i, b]) ^ (t[i, b] * CW[i])
            τ = τ.reshape(2, λ+1)
            s[i+1, b], t[i+1, b] = split(τ[𝛼[i]], [λ, 1])
        
    CW_n = (-1)**t[n, 1]*(beta.to(th.uint8) - Convert(s[n, 0]) + Convert(s[n, 1]))
    
    return s[0, 0], s[0, 1], CW, CW_n
        

In [6]:
@sy.func2plan()
def EvalDPF(b, x, *k_b):
    x = bit_decomposition(x)
    s, t = Array(n+1, λ), Array(n+1, 1)
    s[0] = k_b[0]
    CW =  k_b[1:]
    t[0] = b
    for i in range(0, n):
        τ = G(s[i]) ^ (t[i]*CW[i])
        τ = τ.reshape(2, λ+1)
        s[i+1], t[i+1] = split(τ[x[i]], [λ, 1])
    return (-1)**b * (Convert(s[n]) + t[n]*CW[n])
        

In [7]:
beta = th.IntTensor([2])
alpha, = th.randint(0, 2**n, (1,))

GenDPF.build(alpha, beta)
res = GenDPF(alpha, beta)
s_00, s_01, CW, CW_n = res
k = [
    (s_00, *CW, CW_n),
    (s_01, *CW, CW_n)
]

In [8]:
EvalDPF.build(th.IntTensor([0]), alpha, *k[0])

In [9]:
serde_plan_simplified = serde._simplify(bob, GenDPF)
GenDPF_back = serde._detail(bob, serde_plan_simplified)

serde_plan_simplified = serde._simplify(bob, EvalDPF)
EvalDPF_back = serde._detail(bob, serde_plan_simplified)

In [11]:
beta = th.IntTensor([2])
alpha, = th.randint(0, 2**n, (1,))

In [12]:
res = GenDPF_back(alpha, beta)
s_00, s_01, CW, CW_n = res
k = [
    (s_00, *CW, CW_n),
    (s_01, *CW, CW_n)
]

In [13]:
(EvalDPF(th.IntTensor([0]), alpha, *k[0]) + EvalDPF(th.IntTensor([1]), alpha, *k[1]))

tensor([2], dtype=torch.int32)

In [13]:
(EvalDPF_back(th.IntTensor([0]), alpha+1, *k[0]) + EvalDPF_back(th.IntTensor([1]), alpha+1, *k[1]))

tensor([0], dtype=torch.int32)

In [14]:
def test_DPF():
    t = time.time()
    n_alpha, n_x = 10, 10
    beta = th.IntTensor([2])
    for alpha in th.randint(0, 2**n, (n_alpha, )):
        s_00, s_01, CW, CW_n = GenDPF(alpha, beta)
        k = [
            (s_00, *CW, CW_n),
            (s_01, *CW, CW_n)
        ]
        t2 = time.time()
        for x in th.randint(0, 2**n, (n_x,)):
            y0 = EvalDPF(th.IntTensor([0]), x, *k[0])
            y1 = EvalDPF(th.IntTensor([1]), x, *k[1])
            
            #print(y0+y1)
            if x == alpha:
                assert y0+y1 == beta
            else:
                assert y0+y1 == 0
              
        #print((EvalDPF(0, k[0], alpha) + EvalDPF(1, k[1], alpha)))
        assert (EvalDPF(0, alpha, *k[0]) + EvalDPF(1, alpha, *k[1])) == beta
        
        print(round(1000*(time.time() - t2)/((n_x+1) * 2), 1), 'ms / Eval')
    print(round(1000*(time.time() - t)/(n_alpha*n_x), 1), 'ms')
                
test_DPF()

6.1 ms / Eval
4.4 ms / Eval
4.8 ms / Eval
4.5 ms / Eval
5.2 ms / Eval
4.9 ms / Eval
4.9 ms / Eval
5.8 ms / Eval
5.3 ms / Eval
4.5 ms / Eval
13.9 ms


# DIF

In [15]:
def G(seed):
    assert len(seed) == λ
    th.manual_seed(Convert(seed))
    return th.randint(2, size=(2 + 2*(λ + 1), ), dtype=th.uint8)
    
    
def TruthTableDIF(s, α_i):
    leafTable = th.zeros((2, 1), dtype=th.uint8)
    # if α_i is 0, then ending on the leaf branch means your bit is 1 to you're > α so you should get 0
    # if α_i is 1, then ending on the leaf branch means your bit is 0 to you're < α so you should get 1
    leaf_value = α_i
    leafTable[1-α_i] = leaf_value
    
    nextTable = th.zeros((2, λ+1), dtype=th.uint8)
    one = th.tensor([1], dtype=torch.uint8)
    nextTable[α_i] = concat(s, one)
    
    return concat(leafTable, nextTable, axis=1).flatten()

In [31]:
@sy.func2plan()
def GenDIF(alpha):
    α = bit_decomposition(alpha)
    s, t, CW = Array(n+1, 2, λ), Array(n+1, 2), Array(n, 2 + 2*(λ+1))
    s[0] = randbit(size=(2, λ))
    t[0] = th.tensor([0, 1], dtype=torch.uint8)
    for i in range(0, n):
        # Re-use useless randomness
        _, _, sL_0, _, sR_0, _ = split(G(s[i, 0]), [1, 1, λ, 1, λ, 1])
        _, _, sL_1, _, sR_1, _ = split(G(s[i, 1]), [1, 1, λ, 1, λ, 1])
        s_rand = (sL_0 ^ sL_1) * α[i] + (sR_0 ^ sR_1) * (1 - α[i])
        cw_i = TruthTableDIF(s_rand, α[i])
        CW[i] = cw_i ^ G(s[i, 0]) ^ G(s[i, 1])
        
        for b in (0, 1):
            τ = G(s[i, b])  ^  (t[i, b] * CW[i])
            τ = τ.reshape(2, λ+2)
            σ_leaf, s[i+1, b], t[i+1, b] = split(τ[𝛼[i]], [1, λ, 1])
    
    return s[0, 0], s[0, 1], CW

In [32]:
@sy.func2plan()
def EvalDIF(b, x, *k_b):
    FnOutput = Array(n+1, 1)
    x = bit_decomposition(x)
    s, t = Array(n+1, λ), Array(n+1, 1)
    s[0] = k_b[0]
    CW =  k_b[1:]
    t[0] = b
    for i in range(0, n):
        τ = G(s[i]) ^ t[i]*CW[i]
        τ = τ.reshape(2, λ+2)
        σ_leaf, s[i+1], t[i+1] = split(τ[x[i]], [1, λ, 1])
        FnOutput[i] = σ_leaf

    # Last tour, the other σ is also a leaf:
    FnOutput[n] = t[n]
    return FnOutput.sum() % 2

In [33]:
alpha, = th.randint(0, 2**n, (1,))

GenDIF.build(alpha)
res = GenDIF(alpha)
s_00, s_01, CW = res
k = [
    (s_00, *CW),
    (s_01, *CW)
]

In [34]:
EvalDIF.build(th.IntTensor([0]), alpha, *k[0])

In [35]:
serde_plan_simplified = serde._simplify(bob, GenDIF)
GenDIF_back = serde._detail(bob, serde_plan_simplified)

serde_plan_simplified = serde._simplify(bob, EvalDIF)
EvalDIF_back = serde._detail(bob, serde_plan_simplified)

In [36]:
beta = th.IntTensor([2])
alpha, = th.randint(0, 2**n, (1,))

In [38]:
res = GenDIF_back(alpha, beta)
s_00, s_01, CW = res
k = [
    (s_00, *CW),
    (s_01, *CW)
]

In [42]:
(EvalDIF(th.IntTensor([0]), alpha-1, *k[0]) + EvalDIF(th.IntTensor([1]), alpha-1, *k[1]))

tensor(1)